In [4]:
import pandas as pd
import numpy as np
import pickle

import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import svm, grid_search


/home/kim/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/kim/anaconda3/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [ ]:
df_A = pd.read_csv('data_A.csv', encoding='latin1')
print(df_A.shape)
df_A.dtypes
df_A['description'] = df_A['description'].astype('str')

In [6]:
with open('wf_label.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    preds = pickle.load(f)
with open('clf.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    clf = pickle.load(f)
with open('trigram_LR.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    trigram_LR = pickle.load(f)


    
with open('train_X.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    train_X = pickle.load(f)
with open('train_y.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    train_y = pickle.load(f)
    
with open('X.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    X_train = pickle.load(f)
with open('y.pickle', 'rb') as f:
    # The protocol version used isdetected automatically, so we do not
    # have to specify it.
    y_train = pickle.load(f)

In [7]:
print(list(df_A))

['Unnamed: 0', 'property_id', 'num_bathrooms', 'listing_status', 'listing_id', 'num_bedrooms', 'num_recepts', 'last_marketed_date', 'description', 'property_type', 'price', 'num_floors', 'first_marketed_date', 'property_number', 'postcode', 'address', 'oseast1m', 'osnrth1m', 'laua', 'msoa11', 'lat', 'long']


In [8]:
df_A.drop(['Unnamed: 0'],axis=1, inplace=True)

In [ ]:
df_A.head()

If the id from the labelled dataset it in the WF data, remove it

In [ ]:
preds.head()

In [11]:
print(trigram_LR.fit(X_train, y_train))

Pipeline(steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10000, min_df=1,
        ngram_range=(1, 3), preprocessor=None, stop_words=None,
      ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])


# Create predictions
# 1. get text in right format

In [12]:
def removeStopWords(input):
    exclude = set(string.punctuation)
    output = ' '.join([word for word in input.split() if word not in stopwords.words("english")])
    output = ''.join(ch for ch in output if ch not in exclude)
        
    return pd.Series(dict(output=output))

df_A['ml_text'] = df_A['description'].apply(lambda x: removeStopWords(x))

In [ ]:
df_A['ml_text'].head()

In [14]:
def remove_non_ascii (text):
    return ''.join(i for i in text if ord(i)<128)
                   
def replacenon_ascii_w_space (text):
    return ''.join([i if ord(i) < 128 else ' ' for i in text])
#df['test'] = df['ml_text'].apply(lambda x : x.split(" "))
#df['test'].apply(filter(lambda x : ' '.join([nonAscii(y) for y in x]))
## No success. 

df_A['ml_text'] = df_A['ml_text'].apply(lambda x: remove_non_ascii(x))

In [15]:
# Stem and lemmatize
porter_stemmer = PorterStemmer()
wnl = WordNetLemmatizer()

In [16]:
df_A['token_text'] = df_A['ml_text'].apply(lambda x : x.split(" "))
df_A['stem_text'] = df_A['token_text'].apply(lambda x : [porter_stemmer.stem(y) for y in x])
df_A['stem_text'] = df_A['stem_text'].apply(lambda x : ' '.join(x))
# unseen_wf.drop(['Unnamed: 0', 'description_parsed', 'text', 'token_text', 'ml_text'], axis=1, inplace=True)


In [17]:
with open('df_A.pickle', 'wb') as f:
    pickle.dump(df_A,f,pickle.HIGHEST_PROTOCOL)

In [18]:
X=df_A['stem_text'].values.astype('U')

In [19]:
X1, X2, X3, X4, X5 = np.split(df_A['stem_text'], 5)

with open('X1.pickle', 'wb') as f:
    pickle.dump(X1,f,pickle.HIGHEST_PROTOCOL)

In [20]:
y_pred_prob = trigram_LR.predict_proba(X)[:,1]
LR_output = pd.DataFrame({'X':X, 'predicted':y_pred_prob, 'listing_id':df_A['listing_id'], 'lat':df_A['lat'], 'long':df_A['long'] })
sout = LR_output.sort_values(['X', 'predicted'], ascending=[False, False])
sout.to_html("LRCLF_A_test.html")

In [21]:
trigram_LR.fit(X_train, y_train)
predictions_1 = trigram_LR.predict(X1)
predictions_2 = trigram_LR.predict(X2)
predictions_3 = trigram_LR.predict(X3)
predictions_4 = trigram_LR.predict(X4)
predictions_5 = trigram_LR.predict(X5)

In [22]:
pred_prob_1= trigram_LR.predict_proba(X1)[:,1]
pred_prob_2= trigram_LR.predict_proba(X2)[:,1]
pred_prob_3= trigram_LR.predict_proba(X3)[:,1]
pred_prob_4= trigram_LR.predict_proba(X4)[:,1]
pred_prob_5= trigram_LR.predict_proba(X5)[:,1]

In [ ]:
predictions

with open('predictions_1.pickle', 'wb') as f:
    pickle.dump(predictions_1,f,pickle.HIGHEST_PROTOCOL)
with open('predictions_2.pickle', 'wb') as f:
    pickle.dump(predictions_2,f,pickle.HIGHEST_PROTOCOL)
with open('predictions_3.pickle', 'wb') as f:
    pickle.dump(predictions_3,f,pickle.HIGHEST_PROTOCOL)

with open('pred_prob_1.pickle', 'wb') as f:
    pickle.dump(pred_prob_1,f,pickle.HIGHEST_PROTOCOL)
with open('pred_prob_2.pickle', 'wb') as f:
    pickle.dump(pred_prob_2,f,pickle.HIGHEST_PROTOCOL)
with open('pred_prob_3.pickle', 'wb') as f:
    pickle.dump(pred_prob_3,f,pickle.HIGHEST_PROTOCOL)

id_A= df_A['listing_id'].values.astype('U')

In [ ]:
#udprn = unseen_wf['udprn'].values.astype('U')

In [23]:
# Need to concat the differenct X and predictions arrays and put into one array for each
X_con = np.concatenate((X1,X2,X3, X4, X5),axis=0)
predictions = np.concatenate((predictions_1, predictions_2, predictions_3, predictions_4, predictions_5),axis=0)
pred_prob = np.concatenate((pred_prob_1, pred_prob_2, pred_prob_3,pred_prob_4, pred_prob_5),axis=0)

In [24]:
X_LR_output = pd.DataFrame({'X' :X_con, 'predictions':predictions, 'pred_prob' :pred_prob, 'listing_id' :df_A['listing_id'],'lat':df_A['lat'], 'long':df_A['long']})
sout = X_LR_output.sort_values(['pred_prob', 'predictions'], ascending=[False, False])
sout.to_csv("LR_clf_pred_A.csv")

In [ ]:
len(predictions)

In [26]:
with open('LR_predictions_A.pickle', 'wb') as f:
    pickle.dump(X_LR_output,f,pickle.HIGHEST_PROTOCOL)

In [ ]:
X_LR_output.tail(100)

In [28]:
gate = X_LR_output[X_LR_output['predictions']==1]

In [29]:
with open('gate_A_pred.pickle', 'wb') as f:
    pickle.dump(gate,f,pickle.HIGHEST_PROTOCOL)

In [2]:
import pickle
import pandas as pd
with open('gate_A_pred.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    gate = pickle.load(f)

In [30]:
gate.to_csv('gate_pred_A.csv')

In [21]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
style.use("ggplot")
from sklearn.cluster import KMeans

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
count_vect = TfidfVectorizer()
X_transformed = count_vect.fit_transform(X_train)

In [ ]:
kmeans = KMeans(n_clusters=2)
kmeans.fit_transform(X_transformed)

centroids = kmeans.cluster_centers_
labels = kmeans.labels_

print(centroids)
print(labels)

In [ ]:
colors = ["g.","r.","c.","y."]

for i in range(len(X_transformed)):
    print("coordinate:",X_transformed[i], "label:", labels[i])
    plt.plot(X_transformed[i][0], X_transformed[i][1], colors[labels[i]], markersize = 10)


plt.scatter(centroids[:, 0],centroids[:, 1], marker = "x", s=150, linewidths = 5, zorder = 10)

plt.show()